In [ ]:
import re
import psutil
import torch
from dataclasses import dataclass, field
from typing import Optional


def extract_model_size(base_model: str) -> int:
    """
    Extract the model size in number of parameters from a base_model string.
    The function looks for the last number before 'B' and interprets it as billions of parameters.
    """
    if "B" not in base_model:
        raise ValueError(f"Could not find 'B' in: {base_model}")
    
    before_b = base_model.rsplit("B", 1)[0]
    
    parts = re.split(r'[^0-9._]', before_b)
    numbers = [p for p in parts if p.strip()]
    last_num_str = numbers[-1].replace("_", ".").replace("-", ".")
    
    num_val = float(last_num_str)
    return int(num_val * 1_000_000_000)


@dataclass
class TrainConfig:
    # Model parameters
    base_model: str = "meta-llama/Llama-3.2-1B"
    model_type: str = field(init=False)
    batch_size: int = 2
    quantization: bool = False
    model_size: Optional[int] = None
    
    # Hardware
    device: Optional[str] = None  # e.g., "cuda:0"

    # General
    operator: str = "delta_rule"
    bidirectional: bool = False
    max_chunk_size: int = 64
    linear_precision: str = "bfloat16"
    base_attention_scaling: bool = False

    # MAG parameters
    n_transition_steps: int = 10
    initial_mag: float = 0.0
    mag_ratio: float = 0.5
    base_scale: bool = False
    mag_mode: str = "gradual"
    cross_gate: bool = False

    # LoRA parameters
    lora: bool = True
    lora_rank: int = 8
    lora_alpha: int = 16
    lora_dropout: float = 0.05

    # Training hyperparameters
    learning_rate: float = 5e-4
    weight_decay: float = 0.0
    label_smooth: float = 0.0
    accumulation_grad_step: int = 1
    epoch: int = 1

    # Dataset parameters
    n_samples: int = 100
    test_eval_percent: int = 2
    dataset: str = "yahma/alpaca-cleaned"
    max_seq_len: int = 256

    # Hugging Face Hub parameters
    hf_token: Optional[str] = None
    username: str = "ffurfaro"
    repo_name: str = "Titans-v2-model"

    def __post_init__(self):
        # Automatically set model_type based on base_model string
        if self.model_type is None :
            if "mistral" in self.base_model.lower():
                self.model_type = "mistral"
            else:
                self.model_type = "classic"

        self.model_size = extract_model_size(self.base_model) if self.model_size is None else self.model_size



MODEL_TYPE = ["classic", "moe", "mistral"]

DTYPE_SIZE = {
    torch.float32: 4,
    torch.float16: 2,
    torch.bfloat16: 2,
}

def estimate_memory_mistral(batch_size: int, model_size: int, quantization: bool):
    """
    Estimates GPU memory (GB) needed for Mistral fine-tuning (16-bit),
    Slidding Windows Attention (SWA) change memory calculation
    """
    # Reference base fine-tuning memory for Mistral 7B
    base_ft_mem_gb = 52.0  # average between 50 and 55 GB

    # Activation + optimizer overhead per batch (estimated higher than inference)
    activation_per_batch_gb = 7.0  # rough estimate for fine-tuning

    # Scale base memory by model size proportionally (default 7B)
    base_mem_scaled = base_ft_mem_gb * (model_size / 7_000_000_000)

    # Total memory estimate
    total_mem = base_mem_scaled + batch_size * activation_per_batch_gb

    # Quantization reduces memory roughly by factor 4
    if quantization:
        total_mem *= 0.25

    return total_mem

def estimate_memory_usage(cfg: TrainConfig):
    """
    Estimate GPU memory (GB) needed for training given config, 
    compares to device availability and suggests adjustments if needed.
    """

    model_size = cfg.model_size
    quantization = cfg.quantization
    batch_size = cfg.batch_size
    seq_len = cfg.max_seq_len

    if cfg.model_type == "mistral":
        total_mem_gb = estimate_memory_mistral(batch_size, model_size, quantization)
    else:
        # Generic estimation for other models based on params, dtype, batch, seq_len
        dtype = torch.bfloat16 if not quantization else torch.float16
        param_mem_bytes = model_size * DTYPE_SIZE[dtype]

        activation_mem_bytes = batch_size * seq_len * 4 * DTYPE_SIZE[dtype] * 20  # rough estimate

        total_mem_bytes = param_mem_bytes + activation_mem_bytes

        # Extra 20% overhead for optimizer states, LoRA, etc.
        optimizer_overhead = param_mem_bytes * 1.2
        total_mem_bytes += optimizer_overhead

        total_mem_gb = total_mem_bytes / (1024 ** 3)
    
    if cfg.bidirectional:
        # Bidirectional models typically double the memory usage
        total_mem_gb *= 2
    if "delta_product" in cfg.operator:
        # Delta product operator may increase memory usage
        total_mem_gb *= 2

    # Detect available memory on device
    if torch.cuda.is_available() and "cuda" in cfg.device:
        try:
            gpu_index = int(cfg.device.split(":")[1])
        except Exception:
            gpu_index = 0
        props = torch.cuda.get_device_properties(gpu_index)
        available_mem_gb = props.total_memory / (1024 ** 3)
        device_name = props.name
    else:
        mem = psutil.virtual_memory()
        available_mem_gb = mem.available / (1024 ** 3)
        device_name = "CPU"


    print(f"🔍 Memory estimation for {cfg.model_type} (batch size {batch_size})")
    print(f"   Quantization    : {quantization}")
    print(f"   Device          : {device_name}")
    print(f"   Required memory : {total_mem_gb:.2f} GB")
    print(f"   Available memory: {available_mem_gb:.2f} GB")

    suggestions = []
    if total_mem_gb > available_mem_gb:
        suggestions.append("Reduce batch size (by half or more)")
        suggestions.append("Enable quantization (--quantization True)")
        suggestions.append("Use a smaller model")
        suggestions.append("Use a different hardware (GPU with more memory)")

    if suggestions:
        print("\n⚠️ Not enough memory. Suggestions:")
        for s in suggestions:
            print(f"  - {s}")
    else:
        print("✅ Memory configuration OK.")

    return total_mem_gb, available_mem_gb, suggestions


In [23]:
def extract_model_size(base_model: str) -> int:
    """
    Extract the model size in number of parameters from a base_model string.
    The function looks for the last number before 'B' and interprets it as billions of parameters.
    """
    if "B" not in base_model:
        raise ValueError(f"Could not find 'B' in: {base_model}")
    
    before_b = base_model.rsplit("B", 1)[0]
    
    parts = re.split(r'[^0-9._]', before_b)
    numbers = [p for p in parts if p.strip()]
    last_num_str = numbers[-1].replace("_", ".").replace("-", ".")
    
    num_val = float(last_num_str)
    return int(num_val * 1_000_000_000)


# Tests
print(extract_model_size("meta-llama/Llama-3.2-1B"))  # 1_000_000_000
print(extract_model_size("meta-llama/Llama-1_3B"))    # 1_300_000_000
print(extract_model_size("mistralai/Mistral-7.1B"))     # 7_000_000_000
print(extract_model_size("something3B-13B"))           # 13_000_000_000

1000000000
1300000000
7100000000
13000000000


In [1]:
import typer

app = typer.Typer(add_completion=False)

@app.command()
def check_memory(
    model_type: str = "mistral",
    model_size: int = 7_000_000_000,
    batch_size: int = 2,
    quantization: bool = True,
    device: str = "cpu", #"cuda:0",
):
    """
    Check GPU memory needed for fine-tuning a given model and suggest adjustments.
    """
    cfg = TrainConfig(
        model_type=model_type,
        batch_size=batch_size,
        quantization=quantization,
        device=device,
        model_size=model_size
    )
    estimate_memory_usage(cfg)

if __name__ == "__main__":
    check_memory()

NameError: name 'TrainConfig' is not defined